In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [15]:
# Setting up NNet
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
import matplotlib.pyplot as plt
import hickle as hkl

class NNet(nn.Module):
    def __init__(self):
        super(NNet, self).__init__()
        self.fc1 = nn.Linear(2,10) # H1: 15 or 5 neurons ip -> H1
        self.fc2 = nn.Linear(10,1) # H2: 5-10 or 10-15 neurons
        #self.fc3 = nn.Linear(15,1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))   #later torch.tanh
        x = self.fc2(x)
        return x

def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.uniform_(m.weight.data)
        m.bias.data.fill_(0.01)

net = NNet()
net.apply(weights_init)
print(list(net.parameters()))

[Parameter containing:
tensor([[0.2385, 0.1702],
        [0.8400, 0.3840],
        [0.4616, 0.4337],
        [0.4987, 0.2195],
        [0.3487, 0.6057],
        [0.7293, 0.5100],
        [0.3479, 0.5527],
        [0.1564, 0.5459],
        [0.3791, 0.4533],
        [0.1475, 0.0705]], requires_grad=True), Parameter containing:
tensor([0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100], requires_grad=True), Parameter containing:
tensor([[0.7392, 0.8113, 0.2613, 0.5391, 0.2756, 0.5368, 0.3856, 0.4632, 0.4135,
         0.5451]], requires_grad=True), Parameter containing:
tensor([0.0100], requires_grad=True)]


In [ ]:
# Loading training and validation data to Dataloader
data = hkl.load('gaussian_mixture_set_train_1000.hkl')

coords_list = list(np.transpose(data['coords']))
labels_list = list(np.transpose(data['labels']))
tensor_coord = torch.Tensor(coords_list) # transform to torch tensor
tensor_labels = torch.tensor(labels_list, dtype=torch.long)
gm_trainset = torch.utils.data.TensorDataset(tensor_coord,tensor_labels)
trainloader = torch.utils.data.DataLoader(gm_trainset, batch_size=20, shuffle=True, num_workers=2)



validation_data = hkl.load('gaussian_mixture_set_validation_100.hkl')
coords_list = list(np.transpose(validation_data['coords']))
labels_list = list(np.transpose(validation_data['labels']))
tensor_coord = torch.Tensor(coords_list) # transform to torch tensor
tensor_labels = torch.tensor(labels_list, dtype=torch.long)
gm_validset = torch.utils.data.TensorDataset(tensor_coord,tensor_labels)
validloader = torch.utils.data.DataLoader(gm_validset, batch_size=20, shuffle=False, num_workers=2)


In [4]:
# Defining loss function and optimizer
import torch.optim as optim
criterion = nn.BCEWithLogitsLoss()  #CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
net.train()
max_epoch = 100
max_validation_loss = math.inf
training_loss_check = math.inf

training_loss_mat = np.array([])
validation_loss_mat = np.array([])
for epoch in range(max_epoch):
    
    running_loss = 0.0
    for i, iter_data in enumerate(trainloader, 0):
        # get the inputs
        net.train()
        inputs, labels = iter_data
        inputs = inputs
        labels = labels.flatten()
        optimizer.zero_grad()
        
        # forward + backward + optimize    
        outputs = net(inputs)
        outputs = outputs.flatten()
        labels = labels.type_as(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        
        # print statistics
        running_loss += loss.item()
    
    # validation loss
    validation_loss = 0.0
    for i, iter_data in enumerate(validloader,0):
        net.eval()
        inputs,labels = iter_data
        inputs = inputs
        labels = labels
        labels = labels.flatten()
        
        outputs = net(inputs)
        outputs = outputs.flatten()
        labels = labels.type_as(outputs)
        loss = criterion(outputs, labels)
        validation_loss += loss.item() * inputs.size(0)
        
        
    #print('Completed Epochs:%d  ## Training Loss: %0.3f ## Validation Loss: %0.3f' % (epoch+1, running_loss, validation_loss))
    training_loss_mat = np.append(training_loss_mat,running_loss)
    validation_loss_mat = np.append(validation_loss_mat,validation_loss)
print('Finished Training')

Finished Training


In [6]:
# Testing of the network
test_data = hkl.load('gaussian_mixture_set_test_200.hkl')
coords_list = list(np.transpose(test_data['coords']))
labels_list = list(np.transpose(test_data['labels']))
tensor_coord = torch.Tensor(coords_list) # transform to torch tensor
tensor_labels = torch.tensor(labels_list, dtype=torch.long)
gm_testset = torch.utils.data.TensorDataset(tensor_coord,tensor_labels)
testloader = torch.utils.data.DataLoader(gm_testset, batch_size=20, shuffle=False, num_workers=2)
dataiter = iter(testloader)


error_0 = 0 # True = class-0 , predicted = class-1
error_1 = 0 # True = class-1, predicted = class-0
correct = 0
total = 0
net.eval() #with torch.no_grad(), enables certain dropout layers
with torch.no_grad():
    for i, data in enumerate(testloader,0):
        inputs, labels = data
        inputs = inputs
        labels = labels
        outputs = net(inputs)
        #outputs = net(inputs.float())
        outputs = (torch.sigmoid(outputs)>=0.5).long()
        
        for j in range(outputs.size(0)):
            if outputs.data[j]==0 and labels[j]==1:
                error_1 +=1
            elif outputs.data[j]==1 and labels[j]==0:
                error_0 +=0
        
        total += labels.size(0)
        #print(outputs)
        #print(labels)
        correct += (outputs == labels).sum().item()

print('Total samples tested', total)
error_0 = total-correct-error_1
print("Overall Error: " ,1-(correct/total))
print('Error Conditional Class-0: %0.3f  Class-1: %0.3f'%(error_0/200, error_1/200))
#print(total-correct,total-correct-error_0,error_0)

Total samples tested 400
Overall Error:  0.5
Error Conditional Class-0: 1.000  Class-1: 0.000


In [6]:
#torch.save(net.state_dict(), './saved/model_Tr_600_h1_50_2.pt')

In [9]:
#net = NNet()
#net.load_state_dict(torch.load('./saved/model_Tr_1000_h1_100_3.pt'))
#net.eval()

NNet(
  (fc1): Linear(in_features=2, out_features=15, bias=True)
  (fc2): Linear(in_features=15, out_features=1, bias=True)
)

In [16]:
plt.plot(np.arange(1,101),training_loss_mat,c="b",linewidth=1.5, label="Training Loss")
plt.plot(np.arange(1,101),validation_loss_mat,c="r",linewidth=1.5, label="Validation Loss")
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('No. of Epochs')
plt.ylabel('Loss')
#plt.savefig("./saved/graphs/learning_100_m05.eps",dpi=300)

In [7]:
grid_points = 50
xc = np.linspace(-8,12,grid_points)
yc = np.linspace(-6,6,grid_points)
grid_x, grid_y = np.meshgrid(xc,yc)
coords = np.array((grid_x.ravel(),grid_y.ravel())).T


prediction = np.zeros(xc.shape[0]) 

# Testing of the network to generate contour plot
testloader = torch.utils.data.DataLoader(coords, batch_size=len(coords), shuffle=False, num_workers=2)
dataiter = iter(testloader)


correct = 0
total = 0
net.eval() #with torch.no_grad(), enables certain dropout layers
with torch.no_grad():
    for i, data in enumerate(testloader,0):
        #print(data)
        inputs = data
        #inputs = inputs
        #labels = labels
        outputs = net(inputs.float())
        prediction = (torch.sigmoid(outputs)>=0.5).long().numpy()
        
        total += labels.size(0)
        #print(outputs)
        #print(labels)
        #correct += (outputs == labels).sum().item()

In [ ]:
# Plotting Decision Boundary

prediction = prediction.reshape(grid_points,grid_points)
#xc = xx.reshape(grid_points,grid_points)
#yc = yy.reshape(grid_points,grid_points)
# pp = np.zeros((400,400))
cmap = plt.cm.get_cmap("summer")
plt.contourf(grid_x,grid_y,prediction, cmap=cmap)

zero_ind = np.where(test_data['labels']==0)[1]
one_ind = np.where(test_data['labels']==1)[1]

plt.scatter(test_data['coords'][0,zero_ind],test_data['coords'][1,zero_ind],label = 'Class 0')
plt.scatter(test_data['coords'][0,zero_ind],test_data['coords'][1,one_ind],label = 'Class 1')


plt.xlabel('X [1]')
plt.ylabel('X [2]')
plt.legend(loc = 'upper left',fontsize=14)


plt.xlim(-8,12)
plt.ylim(-6,6)
fig = plt.gcf()
fig.set_size_inches(6.5, 3.5)
#plt.savefig("./saved/graphs/nnet_decision_1000_h2_10_15_50_2.eps",dpi=300)
